In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)

In [4]:
events = pd.read_csv("data/events.csv")
events.head()

,event_id,user_id,product_id,event_type,event_timestamp
0,E00000001,U009798,P001393,cart,2025-07-08T14:28:55.893919
1,E00000002,U005881,P000669,view,2025-10-19T23:00:44.067982
2,E00000003,U006348,P001404,view,2025-05-09T07:02:42.256662
3,E00000004,U002664,P000400,cart,2025-07-19T22:47:07.019634
4,E00000005,U005776,P000392,view,2024-10-24T10:20:33.602165


In [5]:
events.info()
events['event_type'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   event_id         80000 non-null  object
 1   user_id          80000 non-null  object
 2   product_id       80000 non-null  object
 3   event_type       80000 non-null  object
 4   event_timestamp  80000 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


event_type
view        56013
cart        12035
wishlist     7946
purchase     4006
Name: count, dtype: int64

In [6]:
events['event_timestamp']=pd.to_datetime(events['event_timestamp'])

In [7]:
events.dtypes

event_id                   object
user_id                    object
product_id                 object
event_type                 object
event_timestamp    datetime64[ns]
dtype: object

In [8]:
events=events.drop_duplicates()
events=events.dropna(subset=['user_id','event_type','event_timestamp'])

In [9]:
events['event_type'].value_counts()

event_type
view        56013
cart        12035
wishlist     7946
purchase     4006
Name: count, dtype: int64

In [10]:
events=events[events['event_type'].isin(['view','cart','purchase'])]

In [11]:
events=events.sort_values(by=['user_id','event_timestamp'])
events.head()

,event_id,user_id,product_id,event_type,event_timestamp
72771,E00072772,U000001,P000265,cart,2024-01-13 13:42:05.607547
49521,E00049522,U000001,P001739,view,2025-04-05 09:34:26.206475
45585,E00045586,U000001,P001008,view,2025-04-13 23:22:47.182918
33006,E00033007,U000001,P001855,view,2025-08-20 17:38:34.526890
12113,E00012114,U000001,P001416,view,2025-09-22 09:02:17.315929
